In [1]:
import random
import os
import numpy as np
import pandas as pd
from datetime import datetime
from autogluon.tabular import TabularPredictor

random.seed(42)
os.environ['PYTHONHASHSEED'] = str(42)
np.random.seed(42)

# ==========================================
# 1. 데이터 로드
# ==========================================

train_df = pd.read_csv('../Data/train.csv')
test_df = pd.read_csv('../Data/test.csv')
target = '임신 성공 여부'

# ==========================================
# 2. 파생변수 생성
# ==========================================

def derive_features(df):
    
    elderly_categories = ['만35-37세', '만38-39세', '만40-42세', '만43-44세', '만45-50세']
    df['고령 여부'] = df['시술 당시 나이'].isin(elderly_categories).astype(int)

    age_order = {'만18-34세': 1, '만35-37세': 2, '만38-39세': 3, '만40-42세': 4, '만43-44세': 5, '만45-50세': 6, '알 수 없음': 0}
    df['나이_순서'] = df['시술 당시 나이'].map(age_order)
    df['나이x배아'] = df['나이_순서'] * df['이식된 배아 수']

    df['배아 발달 기간'] = df['배아 이식 경과일'] - df['난자 혼합 경과일']
    df['배아 생성 효율'] = df['저장된 배아 수'] / (df['저장된 신선 난자 수'] + 1e-6)
    df['이식 비중'] = df['이식된 배아 수'] / (df['이식된 배아 수'] + df['저장된 배아 수'])
    
    return df

train_df = derive_features(train_df)
test_df = derive_features(test_df)

# ==========================================
# 3. 모델 학습 설정
# ==========================================

predictor = TabularPredictor(
    label=target, 
    eval_metric='roc_auc',
    path='ag_models_out',
).fit(
    train_data=train_df,
    time_limit=3600,
    presets='best_quality',
    ag_args_fit={'num_gpus': 1},
    num_bag_folds=5,
    refit_full=True
)

# ==========================================
# 4. 예측 (Test Data 활용) - 최종 결과를 확률로 출력 (Positive 클래스에 대한 확률만 추출)
# ==========================================

pred_probs = predictor.predict_proba(test_df)
final_probs = pred_probs.iloc[:, 1]


Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.5.0
Python Version:     3.11.14
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 25.2.0: Tue Nov 18 21:09:40 PST 2025; root:xnu-12377.61.12~1/RELEASE_ARM64_T6000
CPU Count:          10
Pytorch Version:    2.9.1
CUDA Version:       CUDA is not available
GPU Count:          WARNING: Exception was raised when calculating GPU count (AssertionError)
Memory Avail:       7.98 GB / 16.00 GB (49.9%)
Disk Space Avail:   199.36 GB / 460.43 GB (43.3%)
Presets specified: ['best_quality']
Using hyperparameters preset: hyperparameters='zeroshot'
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=5, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether 

In [2]:
# --- 1. 리더보드 (오름차순 정렬) ---
lb = predictor.leaderboard(silent=True)
display(lb.sort_values(by='score_val', ascending=False))

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.740140,roc_auc,14.470595,2226.651618,0.030494,6.774683,2,True,8
1,XGBoost_BAG_L1,0.739200,roc_auc,1.513232,129.197153,1.513232,129.197153,1,True,6
2,NeuralNetTorch_BAG_L1,0.737397,roc_auc,3.899187,1167.517384,3.899187,1167.517384,1,True,7
3,NeuralNetFastAI_BAG_L1,0.736513,roc_auc,1.948087,912.177071,1.948087,912.177071,1,True,5
4,ExtraTreesGini_BAG_L1,0.732784,roc_auc,6.823816,9.073366,6.823816,9.073366,1,True,3
5,ExtraTreesEntr_BAG_L1,0.732760,roc_auc,6.951396,9.464018,6.951396,9.464018,1,True,4
6,RandomForestEntr_BAG_L1,0.731074,roc_auc,7.079595,10.985328,7.079595,10.985328,1,True,2
7,RandomForestGini_BAG_L1,0.730263,roc_auc,6.937955,10.333423,6.937955,10.333423,1,True,1
8,ExtraTreesGini_BAG_L1_FULL,NaN,roc_auc,6.823816,9.073366,6.823816,9.073366,1,True,11
9,RandomForestGini_BAG_L1_FULL,NaN,roc_auc,6.937955,10.333423,6.937955,10.333423,1,True,9


In [3]:
# --- 2. 피처 중요도 ---
fi = predictor.feature_importance(data=train_df.sample(n=min(5000, len(train_df)), random_state=42))
display(fi.sort_values(by='p_value', ascending=True).head(30))

These features in provided data are not utilized by the predictor and will be ignored: ['ID', '불임 원인 - 여성 요인', '불임 원인 - 정자 면역학적 요인']
Computing feature importance via permutation shuffling for 71 features using 5000 rows with 5 shuffle sets...
	389.28s	= Expected runtime (77.86s per shuffle set)
	227.33s	= Actual runtime (Completed 5 of 5 shuffle sets)


In [9]:
# --- 3. 제출 파일 생성 ---
submission = pd.read_csv('../Data/sample_submission.csv')
submission['probability'] = final_probs.values

# 현재 시간 가져오기 (예: 0206_1031)
now = datetime.now().strftime('%m%d_%H%M')
file_name = f"{now}_submission.csv"
submission.to_csv(file_name, index=False)

print(f"학습 및 예측이 완료되었습니다. 결과가 {file_name}에 저장되었습니다.")

학습 및 예측이 완료되었습니다. 결과가 0206_1033_submission.csv에 저장되었습니다.
